Day 4: EDA
----------

In [ ]:
# Loading the necessary libraries and the dataset
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_parquet("../data/processed/flights_features.parquet")

In [ ]:
# Plotting the distribution of is_delayed
plt.figure(figsize=(8, 6))
plot = sns.countplot(x='is_delay', data=df, palette='pastel')
plot.set_title('Distribution of Delayed vs On-Time Flights')
plot.set_xlabel('Is Delayed')
plot.set_ylabel('Count')
total = len(df)
for p in plot.patches:
    count = int(p.get_height())
    pct = count / total * 100
    plot.annotate(f'{count}\n{pct:.1f}%', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='bottom')
plt.show()

# ...existing code...
# get counts and percentages for is_delay
counts = df['is_delay'].value_counts().sort_index()               # counts for 0 and 1
pcts = df['is_delay'].value_counts(normalize=True).sort_index() * 100  # percentages

# inspect or use programmatically
print(counts.to_dict())   # e.g. {0: 12345, 1: 678}
print(pcts.to_dict())     # e.g. {0: 94.5, 1: 5.5}

# access individual percentages
pct_on_time = pcts.get(0, 0)
pct_delayed = pcts.get(1, 0)
print(f"On-time: {pct_on_time:.1f}%, Delayed: {pct_delayed:.1f}%")
# ...existing code...